### Instalar librerías y reiniciar kernel.

In [1]:
pip install mysql-connector

     |████████████████████████████████| 11.9MB 364kB/s 
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl size=247950 sha256=1a543420bb4a9df8ff7ee465da58550193fb7d1a69ab280e4b64b2fcf19b2b32
  Stored in directory: /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175
Successfully built mysql-connector


In [2]:
pip install pandas

### Importaciones

In [13]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import *
import json

### Crear acceso a engine de mysql e inspeccionar.

In [ ]:
engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [ ]:
pd.read_sql("select current_date", con=engine)

In [ ]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

In [ ]:
print(engine.table_names())

# ETL 2

## Extraer

In [10]:
df = pd.read_json('unconfirmed-transactions.json')
df.head()

,txs
0,"{'lock_time': 0, 'ver': 1, 'size': 226, 'input..."
1,"{'lock_time': 0, 'ver': 2, 'size': 225, 'input..."
2,"{'lock_time': 0, 'ver': 1, 'size': 371, 'input..."
3,"{'lock_time': 0, 'ver': 1, 'size': 223, 'input..."
4,"{'lock_time': 0, 'ver': 1, 'size': 226, 'input..."


In [11]:
df.dtypes

txs    object
dtype: object

In [24]:
with open('unconfirmed-transactions.json') as json_file:
    data = json.load(json_file)
  
df = pd.json_normalize(data['txs'])
df.head()

,lock_time,ver,size,inputs,weight,time,tx_index,vin_sz,hash,vout_sz,relayed_by,out,rbf
0,0,1,226,"[{'sequence': 4294967295, 'witness': '', 'prev...",904,1611621833,0,1,dd68f3d9b21711243dbd70d3e9914a49bf19e1daa7088b...,2,0.0.0.0,"[{'spent': False, 'tx_index': 0, 'type': 0, 'a...",NaN
1,0,2,225,"[{'sequence': 4294967295, 'witness': '', 'prev...",900,1611621833,0,1,c1dabfa845375b8a07f187522396702a30728cab10d8f2...,2,0.0.0.0,"[{'spent': False, 'tx_index': 0, 'type': 0, 'a...",NaN
2,0,1,371,"[{'sequence': 4294967295, 'witness': '', 'prev...",821,1611621833,0,1,ee3796f7c2c07fca9b48d04e9ec71becb24f621965b119...,2,0.0.0.0,"[{'spent': False, 'tx_index': 0, 'type': 0, 'a...",NaN
3,0,1,223,"[{'sequence': 4294967295, 'witness': '', 'prev...",892,1611621833,0,1,c28cb5b54db80ace84ec926dc72f1bb4db53b2d30a6f18...,2,0.0.0.0,"[{'spent': False, 'tx_index': 0, 'type': 0, 'a...",NaN
4,0,1,226,"[{'sequence': 4294967295, 'witness': '', 'prev...",904,1611621832,0,1,83fe83d4c709b54694686c0170d50d01f6c05d36ad7a84...,2,0.0.0.0,"[{'spent': False, 'tx_index': 0, 'type': 0, 'a...",NaN


In [25]:
df.dtypes

lock_time      int64
ver            int64
size           int64
inputs        object
weight         int64
time           int64
tx_index       int64
vin_sz         int64
hash          object
vout_sz        int64
relayed_by    object
out           object
rbf           object
dtype: object

## Transformar

In [26]:
df = df.drop(['inputs', 'out', 'rbf'], 1)
df.dtypes

lock_time      int64
ver            int64
size           int64
weight         int64
time           int64
tx_index       int64
vin_sz         int64
hash          object
vout_sz        int64
relayed_by    object
dtype: object

In [29]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

## Load al engine

In [28]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.unconfirmed_transactions', con=engine)
print(create_table_query)
type(create_table_query)

In [ ]:
with open("unconfirmed_transactions.sql", "w") as text_file:
    text_file.write(create_table_query)

In [ ]:
if not engine.dialect.has_table(engine, "test.unconfirmed_transactions"):  # If table don't exist, Create.
    print("test.unconfirmed_transactions no existe")
    df.to_sql('unconfirmed_transactions', con=engine, schema="test", if_exists='append')

In [ ]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.unconfirmed_transactions')
    df.to_sql('unconfirmed_transactions', con=transaction, schema='test', if_exists='append', index=False)